In [ ]:
import ast
from dhanhq import dhanhq, DhanContext
import functions as f

# Your existing code to get credentials
client_id = str(ast.literal_eval(f.get_line('data/database.txt', 3).strip())['client_id'])
access_token = str(ast.literal_eval(f.get_line('data/database.txt', 4).strip())['access_token'])

# Correct v2.1.0 initialization
dhan_context = DhanContext(client_id, access_token)
dhan = dhanhq(dhan_context)

In [ ]:
dhan.get_fund_limits()


In [ ]:
# option chain
import pandas as pd

responce = dhan.option_chain(
    under_security_id=13,                       # Nifty
    under_exchange_segment="IDX_I",
    expiry="2025-06-05"
)
responce

def extract_option_chain_data(response_json):
    oc_data = response_json['data']['data']['oc']
    rows = []

    for strike, data in oc_data.items():
        ce = data.get('ce', {})
        pe = data.get('pe', {})

        row = {
            'strike_price': float(strike),
            'ce_iv': ce.get('implied_volatility', 0),
            'ce_last_price': ce.get('last_price', 0),
            'ce_oi': ce.get('oi', 0),
            'ce_volume': ce.get('volume', 0),
            'pe_iv': pe.get('implied_volatility', 0),
            'pe_last_price': pe.get('last_price', 0),
            'pe_oi': pe.get('oi', 0),
            'pe_volume': pe.get('volume', 0)
        }
        rows.append(row)

    df = pd.DataFrame(rows)
    df.sort_values('strike_price', inplace=True)
    df = df[['ce_iv', 'ce_volume', 'ce_oi', 'ce_last_price', 'strike_price', 'pe_last_price', 'pe_iv', 'pe_oi', 'pe_volume']]


    return df
df = extract_option_chain_data(responce)
df

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# # DhanHQ Order Placement Examples
# # Make sure you have already initialized dhan_context and dhan as shown in your first cell

# # =============================================================================
# # 1. LIMIT ORDER
# # =============================================================================
# def place_limit_order():
#     """Place a limit order"""
#     response = dhan.place_order(
#         security_id='1333',                    # HDFC Bank security ID
#         exchange_segment=dhan.NSE,             # NSE_EQ
#         transaction_type=dhan.BUY,             # BUY or SELL
#         quantity=10,                           # Number of shares
#         order_type=dhan.LIMIT,                 # LIMIT order type
#         product_type=dhan.CNC,                 # CNC (Cash and Carry) or INTRA
#         price=1500.50,                         # Limit price
#         trigger_price=0,                       # Not needed for limit orders
#         disclosed_quantity=0,                  # Optional: disclosed quantity
#         validity=dhan.DAY,                     # DAY or IOC
#         tag="limit_order_001"                  # Optional: correlation ID for tracking
#     )
#     print("Limit Order Response:", response)
#     return response

# # =============================================================================
# # 2. MARKET ORDER
# # =============================================================================
# def place_market_order():
#     """Place a market order"""
#     response = dhan.place_order(
#         security_id='1333',                    # HDFC Bank security ID
#         exchange_segment=dhan.NSE,
#         transaction_type=dhan.BUY,
#         quantity=5,
#         order_type=dhan.MARKET,                # MARKET order type
#         product_type=dhan.INTRA,               # Intraday
#         price=0,                               # Price = 0 for market orders
#         tag="market_order_001"
#     )
#     print("Market Order Response:", response)
#     return response

# # =============================================================================
# # 3. STOP LOSS (SL) ORDER
# # =============================================================================
# def place_stop_loss_order():
#     """Place a stop loss order"""
#     response = dhan.place_order(
#         security_id='1333',
#         exchange_segment=dhan.NSE,
#         transaction_type=dhan.SELL,            # Usually SELL for stop loss
#         quantity=10,
#         order_type=dhan.SL,                    # STOP_LOSS order type
#         product_type=dhan.CNC,
#         price=1450.00,                         # Limit price after trigger
#         trigger_price=1460.00,                 # Trigger price
#         tag="stop_loss_001"
#     )
#     print("Stop Loss Order Response:", response)
#     return response

# # =============================================================================
# # 4. STOP LOSS MARKET (SLM) ORDER
# # =============================================================================
# def place_stop_loss_market_order():
#     """Place a stop loss market order"""
#     response = dhan.place_order(
#         security_id='1333',
#         exchange_segment=dhan.NSE,
#         transaction_type=dhan.SELL,
#         quantity=10,
#         order_type=dhan.SLM,                   # STOP_LOSS_MARKET order type
#         product_type=dhan.CNC,
#         price=0,                               # Price = 0 for market execution
#         trigger_price=1460.00,                 # Trigger price
#         tag="slm_order_001"
#     )
#     print("Stop Loss Market Order Response:", response)
#     return response

# # =============================================================================
# # 5. COVER ORDER (CO)
# # =============================================================================
# def place_cover_order():
#     """Place a cover order"""
#     response = dhan.place_order(
#         security_id='1333',
#         exchange_segment=dhan.NSE,
#         transaction_type=dhan.BUY,
#         quantity=10,
#         order_type=dhan.MARKET,                # Usually MARKET for entry
#         product_type=dhan.CO,                  # Cover Order product type
#         price=0,                               # Market price
#         trigger_price=1400.00,                 # Stop loss trigger price
#         tag="cover_order_001"
#     )
#     print("Cover Order Response:", response)
#     return response

# # =============================================================================
# # 6. BRACKET ORDER (BO)
# # =============================================================================
# def place_bracket_order():
#     """Place a bracket order"""
#     response = dhan.place_order(
#         security_id='1333',
#         exchange_segment=dhan.NSE,
#         transaction_type=dhan.BUY,
#         quantity=10,
#         order_type=dhan.LIMIT,
#         product_type=dhan.BO,                  # Bracket Order product type
#         price=1500.00,                         # Entry price
#         bo_profit_value=50.00,                 # Target profit value
#         bo_stop_loss_Value=25.00,              # Stop loss value
#         tag="bracket_order_001"
#     )
#     print("Bracket Order Response:", response)
#     return response

# # =============================================================================
# # 7. AFTER MARKET ORDER (AMO)
# # =============================================================================
# def place_amo_order():
#     """Place an After Market Order"""
#     response = dhan.place_order(
#         security_id='1333',
#         exchange_segment=dhan.NSE,
#         transaction_type=dhan.BUY,
#         quantity=10,
#         order_type=dhan.LIMIT,
#         product_type=dhan.CNC,
#         price=1500.00,
#         after_market_order=True,               # Enable AMO
#         amo_time='OPEN',                       # 'OPEN', 'OPEN_30', 'OPEN_60'
#         tag="amo_order_001"
#     )
#     print("AMO Order Response:", response)
#     return response

# # =============================================================================
# # 8. MTF (MARGIN TRADING FACILITY) ORDER
# # =============================================================================
# def place_mtf_order():
#     """Place an MTF order"""
#     response = dhan.place_order(
#         security_id='1333',
#         exchange_segment=dhan.NSE,
#         transaction_type=dhan.BUY,
#         quantity=100,
#         order_type=dhan.MARKET,
#         product_type=dhan.MTF,                 # Margin Trading Facility
#         price=0,
#         tag="mtf_order_001"
#     )
#     print("MTF Order Response:", response)
#     return response

# # =============================================================================
# # 9. SLICE ORDER (for large quantities)
# # =============================================================================
# def place_slice_order():
#     """Place a slice order for large quantities"""
#     response = dhan.place_slice_order(
#         security_id='52175',                   # Nifty options
#         exchange_segment=dhan.NSE_FNO,
#         transaction_type=dhan.BUY,
#         quantity=2000,                         # Large quantity that exceeds freeze limit
#         order_type=dhan.MARKET,
#         product_type=dhan.INTRA,
#         price=0,
#         tag="slice_order_001"
#     )
#     print("Slice Order Response:", response)
#     return response

# # =============================================================================
# # USAGE EXAMPLES
# # =============================================================================
# if __name__ == "__main__":
#     # Uncomment the orders you want to place
    
#     # Basic orders
#     # place_limit_order()
#     # place_market_order()
    
#     # Stop loss orders
#     # place_stop_loss_order()
#     # place_stop_loss_market_order()
    
#     # Advanced orders
#     # place_cover_order()
#     # place_bracket_order()
    
#     # Special orders
#     # place_amo_order()
#     # place_mtf_order()
#     # place_slice_order()
    
#     print("Order placement functions are ready to use!")

# # =============================================================================
# # ADDITIONAL UTILITY FUNCTIONS
# # =============================================================================

# def get_order_status(order_id):
#     """Get status of a specific order"""
#     response = dhan.get_order_by_id(order_id)
#     print(f"Order {order_id} Status:", response)
#     return response

# def get_all_orders():
#     """Get all orders for the day"""
#     response = dhan.get_order_list()
#     print("All Orders:", response)
#     return response

# def modify_existing_order(order_id, new_quantity, new_price):
#     """Modify an existing order"""
#     response = dhan.modify_order(
#         order_id=order_id,
#         order_type=dhan.LIMIT,
#         leg_name="ENTRY_LEG",
#         quantity=new_quantity,
#         price=new_price,
#         trigger_price=0,
#         disclosed_quantity=0,
#         validity=dhan.DAY
#     )
#     print(f"Order {order_id} Modified:", response)
#     return response

# def cancel_existing_order(order_id):
#     """Cancel an existing order"""
#     response = dhan.cancel_order(order_id)
#     print(f"Order {order_id} Cancelled:", response)
#     return response

# # =============================================================================
# # COMMON SECURITY IDs FOR REFERENCE
# # =============================================================================
# """
# Common Security IDs:
# - HDFC Bank: 1333 (NSE)
# - Reliance: 2885 (NSE)
# - TCS: 11536 (NSE)
# - Infosys: 1594 (NSE)
# - ICICI Bank: 4963 (NSE)
# - Nifty 50 Index: 13 (NSE)
# - Bank Nifty Index: 25 (NSE)

# Exchange Segments:
# - NSE Equity: dhan.NSE (NSE_EQ)
# - BSE Equity: dhan.BSE (BSE_EQ)
# - NSE F&O: dhan.NSE_FNO (NSE_FNO)
# - BSE F&O: dhan.BSE_FNO (BSE_FNO)
# - Currency: dhan.CUR (NSE_CURRENCY)
# - Commodity: dhan.MCX (MCX_COMM)
# """

In [ ]:
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:17:54'}
# {'type': 'Previous Close', 'exchange_segment': 1, 'security_id': 1333, 'prev_close': '1927.50', 'prev_OI': 0}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:17:55'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:17:56'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:17:57'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:17:58'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:17:58'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:17:59'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:00'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:01'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:02'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:03'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:04'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:05'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:06'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:07'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:08'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:08'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:09'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:10'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:11'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:12'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:13'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:14'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:15'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:16'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:17'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:17'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:18'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:19'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:20'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:21'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:22'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:23'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:24'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:25'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:25'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:26'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:27'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:28'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:29'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:30'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:31'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:32'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:33'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:34'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:35'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:35'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:36'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:37'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:38'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:39'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:40'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:41'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:42'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:43'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:44'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:44'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:45'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:46'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:47'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:48'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:49'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:50'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:51'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:52'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:53'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:53'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:54'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:55'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:56'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:57'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:18:58'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:18:59'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:00'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:01'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:02'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:02'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:03'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:04'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:05'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:06'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:07'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:08'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:09'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:10'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:11'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:11'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:12'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:13'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:14'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:15'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:16'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:17'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:18'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:19'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:20'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:20'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:21'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:22'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:23'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:24'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:25'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:26'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:27'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:28'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:29'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:29'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:30'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:31'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:32'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:33'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:34'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:35'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:36'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:37'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:38'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:39'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:39'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:40'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:41'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:42'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:43'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:44'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:45'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:46'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:47'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:47'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:48'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:49'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:50'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:51'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:52'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:53'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:53'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:55'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:56'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:56'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:57'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:19:58'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:19:59'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:00'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:01'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:02'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:03'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:04'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:05'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:06'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:06'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:07'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:08'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:09'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:10'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:11'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:12'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:13'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:13'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:14'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:15'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:16'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:17'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:18'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:19'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:20'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:21'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:22'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:23'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:23'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:24'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:25'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:26'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:27'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:28'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:29'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:30'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:31'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:32'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:33'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:33'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:34'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:35'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:36'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:37'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:38'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:39'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:40'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:41'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:42'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:42'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:43'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:44'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:45'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:46'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:47'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:48'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:49'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:50'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:50'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:51'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:52'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:53'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:54'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:55'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:56'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:57'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:20:58'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:20:59'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:00'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:00'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:01'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:02'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:03'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:04'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:05'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:06'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:07'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:08'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:08'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:09'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:10'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:11'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:12'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:13'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:14'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:15'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:16'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:17'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:17'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:18'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:19'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:20'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:21'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:22'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:23'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1944.90', 'LTT': '15:21:24'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:25'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:26'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:27'}
# {'type': 'Ticker Data', 'exchange_segment': 1, 'security_id': 1333, 'LTP': '1945.00', 'LTT': '15:21:27'}

In [ ]:
# import nest_asyncio
# import asyncio
# from dhanhq import MarketFeed

# # Enable nested event loops for Jupyter
# nest_asyncio.apply()

# # Structure for subscribing is (exchange_segment, "security_id", subscription_type)
# instruments = [(MarketFeed.NSE, "1333", MarketFeed.Ticker)]
# version = "v2"          # Mention Version and set to latest version 'v2'

# # Method 1: Using asyncio.run() with proper async handling
# async def run_market_feed():
#     try:
#         data = MarketFeed(dhan_context, instruments, version)
#         await data.connect()  # Connect first
        
#         n = 0
#         while n <= 10:
#             response = await data.get_instrument_data()  # Use async method
#             print(response)
#             n += 1
            
#         # Properly await the disconnect
#         await data.disconnect()
        
#     except Exception as e:
#         print("Error:", e)

# # Run the async function
# asyncio.run(run_market_feed())

In [ ]:
import nest_asyncio
import asyncio
from dhanhq import MarketFeed

# Enable nested event loops for Jupyter
nest_asyncio.apply()

# Structure for subscribing is (exchange_segment, "security_id", subscription_type)

instruments = [(MarketFeed.NSE, "1333", MarketFeed.Ticker)]

version = "v2"          # Mention Version and set to latest version 'v2'

# In case subscription_type is left as blank, by default Ticker mode will be subscribed.

try:
    data = MarketFeed(dhan_context, instruments, version)
    
    while True:
        data.run_forever()
        response = data.get_data()
        print(response)

except Exception as e:
    print("Error", e)


In [ ]:
data.disconnect()

In [ ]:
# Subscribe instruments while connection is open
sub_instruments = [(MarketFeed.NSE, "14436", MarketFeed.Ticker)]
data.subscribe_symbols(sub_instruments)

In [ ]:
# Unsubscribe instruments which are already active on connection
unsub_instruments = [(MarketFeed.NSE, "1333", 16)]
data.unsubscribe_symbols(unsub_instruments)

In [ ]:
import nest_asyncio
import asyncio
from dhanhq import MarketFeed # Assuming dhanhq supports callbacks

# Enable nested event loops for Jupyter
nest_asyncio.apply()

# Structure for subscribing is (exchange_segment, "security_id", subscription_type)

instruments = [(MarketFeed.NSE, "1333", MarketFeed.Ticker),
               (MarketFeed.NSE, "11915", MarketFeed.Full)]

version = "v2"

# 1. Define a callback function to handle incoming data
def my_data_handler(market_data):
    """
    This function will be called every time new market data arrives.
    """
    print(f"Time: {market_data.timestamp}, Symbol: {market_data.security_id}, Price: {market_data.ltp}")
    # You would put your trading logic, logging, or display updates here

try:
    data = MarketFeed(dhan_context, instruments, version)

    # 2. Register your callback function with the MarketFeed object
    #    (This is a common pattern; the actual method name might vary, e.g., on_message, set_handler)
    data.on_market_data(my_data_handler) # CONCEPTUAL: Assuming such a method exists

    print("MarketFeed connected and running... Waiting for data (Press Ctrl+C to stop)")
    # 3. Start the event loop. Now, when data arrives, my_data_handler will be called.
    data.run_forever()

except Exception as e:
    print(f"An error occurred: {e}")
except KeyboardInterrupt: # This is how you'd typically stop a run_forever() loop gracefully
    print("\nMarketFeed stopped by user (Ctrl+C).")
    # You might also call data.stop() here if the library provides it for graceful shutdown.

In [ ]:
# Cell 1: Initial Setup and Start Market Feed
import nest_asyncio
import asyncio
from dhanhq import DhanContext, MarketFeed

nest_asyncio.apply()

# Initial instruments
instruments = [
    (MarketFeed.NSE, "1333", MarketFeed.Ticker),
    (MarketFeed.NSE, "1333", MarketFeed.Quote),
    (MarketFeed.NSE, "1333", MarketFeed.Full),
    (MarketFeed.NSE, "11915", MarketFeed.Ticker),
    (MarketFeed.NSE, "11915", MarketFeed.Full)
]

class MarketDataManager:
    def __init__(self, dhan_context, instruments, version="v2"):
        self.market_feed = MarketFeed(dhan_context, instruments, version)
        self.running = False
        self.task = None
        self.subscribed_instruments = set(instruments)  # Track subscriptions
    
    async def start(self):
        if self.running:
            print("Market feed is already running")
            return
            
        self.running = True
        await self.market_feed.connect()
        self.task = asyncio.create_task(self._data_loop())
        print("Market feed started")
        print(f"Initially subscribed to: {len(self.subscribed_instruments)} instruments")
    
    async def _data_loop(self):
        try:
            while self.running:
                response = await self.market_feed.get_instrument_data()
                if response:
                    # Print only essential info to reduce clutter
                    print(f"[{response.get('type', 'Unknown')}] Security: {response.get('security_id')} | LTP: {response.get('LTP', 'N/A')}")
                await asyncio.sleep(0.1)
        except Exception as e:
            print(f"Data loop error: {e}")
    
    async def add_instruments(self, new_instruments):
        """Add new instruments to subscription"""
        try:
            self.market_feed.subscribe_symbols(new_instruments)
            self.subscribed_instruments.update(new_instruments)
            print(f"✅ Added instruments: {new_instruments}")
            print(f"Total subscribed: {len(self.subscribed_instruments)} instruments")
        except Exception as e:
            print(f"❌ Error adding instruments: {e}")
    
    async def remove_instruments(self, instruments_to_remove):
        """Remove instruments from subscription"""
        try:
            self.market_feed.unsubscribe_symbols(instruments_to_remove)
            self.subscribed_instruments.difference_update(instruments_to_remove)
            print(f"✅ Removed instruments: {instruments_to_remove}")
            print(f"Total subscribed: {len(self.subscribed_instruments)} instruments")
        except Exception as e:
            print(f"❌ Error removing instruments: {e}")
    
    def list_subscriptions(self):
        """List all currently subscribed instruments"""
        print(f"Currently subscribed to {len(self.subscribed_instruments)} instruments:")
        for i, instrument in enumerate(self.subscribed_instruments, 1):
            exchange, security_id, sub_type = instrument
            print(f"  {i}. Exchange: {exchange}, Security: {security_id}, Type: {sub_type}")
    
    async def stop(self):
        self.running = False
        if self.task:
            self.task.cancel()
            try:
                await self.task
            except asyncio.CancelledError:
                pass
        await self.market_feed.disconnect()
        print("Market feed stopped")

# Create and start manager
manager = MarketDataManager(dhan_context, instruments)
await manager.start()

In [ ]:
instruments_to_remove = [(MarketFeed.NSE, "1333", MarketFeed.Ticker)]
manager.remove_instruments = (instruments_to_remove)

In [ ]:
new_instruments = [(MarketFeed.NSE_FNO, "1333", MarketFeed.Ticker)]
manager.add_instruments(new_instruments)

In [ ]:
import pandas as pd

# master_list = pd.read_csv("https://images.dhan.co/api-data/api-scrip-master.csv", sep=None, engine='python', on_bad_lines='skip')
# master_list.to_csv('data/master_list.csv', index=False)

master_list = pd.read_csv('data/master_list.csv', sep=None, engine='python', on_bad_lines='skip')

master_12 = master_list.copy()
master_12.rename(columns={'SEM_EXCHANGE': 'SEM_EXM_EXCH_ID', 'SEM_SCRIP_CODE': 'SEM_SMST_SECURITY_ID'}, inplace=True)
master_12 = master_12.drop(['SEM_EXCH_INSTRUMENT_TYPE', 'SEM_SERIES'], axis=1)
master_12[["symbol1", "date1", "month1", "strike1", "CE_PE1"]] = master_12["SEM_CUSTOM_SYMBOL"].str.split(" ", n=4, expand=True)
master_12 = master_12[master_12['strike1'].apply(lambda x: str(x).isnumeric())]
master_12['strike1'] = pd.to_numeric(master_12['strike1'])
master_12 = master_12[['SEM_EXM_EXCH_ID', 'SEM_INSTRUMENT_NAME', 'symbol1', 'month1', 'date1', 'strike1', 'SEM_CUSTOM_SYMBOL', 'SEM_SMST_SECURITY_ID', 'CE_PE1']]



In [ ]:
master_12 = master_12[
    (master_12.date1 == "05") &
    (master_12.month1 == "JUN") &
    (master_12.SEM_INSTRUMENT_NAME == "OPTIDX") &
    (master_12.symbol1 == "NIFTY")]
master_12

In [ ]:
master_12[
    (master_12.strike1 == 18750) &
    (master_12.CE_PE1 == "CALL")]

In [ ]:
for i in range(1, 2000):
    mask = (master_12['strike1'].values == 18750) & (master_12['CE_PE1'].values == 'CALL')
    filtered_ids = master_12['SEM_SMST_SECURITY_ID'].values[mask]
    sem_smst_security_id = filtered_ids[0] if filtered_ids.size > 0 else None
    sem_smst_security_id


In [ ]:
mask = (master_12['strike1'].values == 18750) & (master_12['CE_PE1'].values == 'CALL')
filtered_ids = master_12['SEM_SMST_SECURITY_ID'].values[mask]
sem_smst_security_id = filtered_ids[0] if filtered_ids.size > 0 else None
sem_smst_security_id